## Benötigte Pakete laden

In [1]:
import pandas as pd
import geopandas as gpd
import re

## Übersicht über benötigte Dateien

Annahme: alle relevanten Dateien liegen in dem Unterordner `data`:
- `result.csv` (Quelle: https://open-data.bielefeld.de/dataset/a9442cf1-7f88-4d7e-9fb7-b2a294f533e7/resource/56450c7c-c359-43dd-91e1-14299141200e)
- `trees.geojson` (privat)

In [ ]:
csv_file_path = 'data/result.csv'
geojson_file_path = 'data/trees.geojson'

## Die Daten laden und miteinander vergleichen

In [ ]:
# csv als DataFrame laden
df_oid = pd.read_csv(csv_file_path, sep=";") 
df_oid.head()

In [ ]:
# geojson als DataFrame laden
gdf = gpd.read_file(geojson_file_path)
gdf.head()

In [ ]:
# Anzahl der Einträge miteinander vergleichen
print(f"Anzahl der Einzelbäume in der 'result.csv': {df_oid.shape[0]}")
print(f"Anzahl der Einzelbäume in der 'trees.geojson': {gdf.shape[0]}")

**Problem:** In der `trees.geojson` sind also mehr Einzelbäume enthalten, sodass wir zusätzliche OIDs generieren müssen!

## Koordinaten im DataFrame und GeoDataFrame normalisieren

**Problem:** Die Koordinaten haben unterschiedlich viele Nachkommastellen, deswegen müssen wir sie normalisieren.

In [ ]:
def extract_and_round_coords(wkt, decimals=3):
    match = re.search(r'POINT \(([0-9.]+) ([0-9.]+)\)', wkt)
    if match:
        x = round(float(match.group(1)), decimals)
        y = round(float(match.group(2)), decimals)
        return f"{x} {y}"
    else:
        return None 

df_oid['rounded_coords'] = df_oid['WKT'].apply(lambda x: extract_and_round_coords(x))
df_oid[['WKT', 'rounded_coords']].head()

In [ ]:
def round_coords(x, decimals=3):
    try:
        rounded_x = round(float(x.x), decimals)
        rounded_y = round(float(x.y), decimals)
        return f"{rounded_x} {rounded_y}"
    except (OverflowError, ValueError) as e:
        print(f"Error rounding coordinates: {e}")
        return None
        
gdf['rounded_coords'] = gdf['geometry'].apply(lambda x: round_coords(x))
gdf[['geometry', 'rounded_coords']].head()

## Die `OID`s in das GeoDataFrame einfügen

In [ ]:
# Merge das GeoDataFrame mit dem DataFrame, um OID einzufügen, anhand von 'rounded_coords', mittels einem Left Join
gdf_merged = gdf.merge(df_oid[['rounded_coords', 'oid']], on='rounded_coords', how='left')
gdf_merged.head()

In [ ]:
# Zählen, wie viele Bäume noch keine OID erhalten haben
gdf_merged['oid'].isna().sum()

## Für die nicht-gematchten Bäume `OID`s generieren

In [ ]:
# Ermitteln der höchsten OID im DataFrame
max_oid = df_oid['oid'].max()

In [ ]:
# Bäume noch ohne OID identifizieren
non_matching_gdf = gdf_merged[gdf_merged['oid'].isna()]

# Fortlaufende OIDs zu den nicht übereinstimmenden Bäumen zuweisen
non_matching_gdf = non_matching_gdf.reset_index(drop=True)
non_matching_gdf['oid'] = non_matching_gdf.index + max_oid + 1

In [ ]:
# Aktualisieren der OIDs in gdf_merged
gdf_merged.loc[gdf_merged['oid'].isna(), 'oid'] = non_matching_gdf['oid'].values
gdf_merged.head()

In [ ]:
# Finaler Check: Zählen, wie viele Bäume noch keine OID erhalten haben
gdf_merged['oid'].isna().sum()

## Das GeoDataFrame aufräumen

In [ ]:
# Entfernen der `rounded_coords`-Spalte
gdf_merged = gdf_merged.drop(columns=['rounded_coords'], errors='ignore')
gdf_merged.head()

In [ ]:
# Setzen der `oid`-Spalte als erste Spalte
cols = ['oid'] + [col for col in gdf_merged.columns if col != 'oid']
gdf_merged = gdf_merged[cols]

gdf_merged.head()

In [ ]:
# Konvertieren der OID-Spalte in Ganzzahl, um Dezimalstellen zu vermeiden
gdf_merged['oid'] = gdf_merged['oid'].astype(int)

## Das GeoDataFrame als `geojson` speichern

In [ ]:
# Sicherstellen, dass gdf_merged ein GeoDataFrame ist
gdf_merged = gpd.GeoDataFrame(gdf_merged, geometry='geometry')

# Speichern des kombinierten GeoDataFrames in eine neue GeoJSON-Datei
gdf_merged.to_file('data/trees_with_oids.geojson', driver='GeoJSON')